# Ejercicio 11
Se ha realizado un análisis químico a tres tipos distintos de vinos producidos en una misma región de Italia.
El número de muestras considerado es el siguiente:

- Tipo 1 ➔ 59 muestras
- Tipo 2 ➔ 71 muestras
- Tipo 3 ➔48 muestras

El archivo Vinos.csv permite observar los resultados de este análisis. Cada fila representa una muestra
distinta y está formada, en primer lugar, por el número del tipo al cual pertenece el vino analizado seguido
por los 13 atributos que lo caracterizan.

Por ejemplo, la siguiente fila:
- 2, 12.29, 3.17, 2.21, 18, 88, 2.85, 2.99, 0.45, 2.81, 2.3, 1.42, 2.83, 406


es el resultado del análisis de un vino correspondiente al tipo 2 (1er. valor de la fila) seguido por 13 valores
separados por comas que indican los niveles de las mediciones realizadas a dicho vino.

Entrene una red neuronal formada por una única neurona para clasificar los vinos de Tipo 1.
Realice 30 ejecuciones independientes utilizando el 50%, 60%, 70%, 80% y 90% de los ejemplos como
entrenamiento y el resto como testeo. Para cada porcentaje, indique la cantidad promedio de ejemplos
correctamente clasificados en entrenamiento y en testeo. Calcule también el promedio y el desvío de la
cantidad de iteraciones realizadas.

Utilice un máximo de 400 iteraciones y velocidades de aprendizaje 0.1, 0.2 y 0.3.

Analice los resultados obtenidos utilizando:
- i. Función de activación ‘sigmoid’ y función de costo ‘ECM’ (error cuadrático medio)
- ii. Función de activación ‘sigmoid’ y función de costo ‘EC_binaria’ (entropía cruzada binaria)
- iii. Función de activación ‘tanh’ y función de costo ‘ECM’ (error cuadrático medio)

In [1]:
import pandas as pd
import chardet
import numpy as np
from sklearn import preprocessing, model_selection
from matplotlib import pyplot as plt
from ClassNeuronaGral import NeuronaGradiente

In [2]:
def open_file(path, delimiter=';'):
    # Detectar la codificación del archivo
    with open(path, 'rb') as file:
        encoding = chardet.detect(file.read())['encoding']
    
    # Leer el archivo CSV usando el delimitador correcto
    df = pd.read_csv(path, encoding=encoding, delimiter=delimiter)
    
    return df

df = open_file("../Datos/Vinos.csv")
df

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [3]:
T_raw = df.iloc[:, 0]
X_raw = df.iloc[:, 1:]

clases = np.unique(T_raw)  # obtiene las clases sin repeticiones
print('Las clases del dataset son :', clases)

# Normalizacion con media y desviacion
scaler = preprocessing.StandardScaler()
# scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X_raw)

Las clases del dataset son : [1 2 3]


In [10]:
TABLA = [["Funcion", "Alfa", "Costo", "TEST_SIZE", "Iteraciones","Accuracy train","Veces (datos Train)", "Accuracy test", "Veces (Datos Test)"]]

for FUN in ['sigmoid', 'tanh']:
    for ALFA in [0.1, 0.2, 0.3]:
        for COSTO in ['ECM', 'EC_binaria']:
            for TEST_SIZE in [.5, .6, .7, .8, .9]:
            
                ERROR = 1e-03
                MAX_ITE = 400
                CORRIDAS = 30
                
                # print('\nNeurona con %s' %FUN)
                
                # hace entrenamiento por cada clase
                cls_name = 1
                prom_ite = 0
                prom_acc_train = 0
                prom_acc_test = 0
                veces_train = 0
                veces_test = 0
                if FUN=='sigmoid':
                    T = 1*(T_raw==cls_name)
                else:
                    T = 2*(T_raw==cls_name)-1
          
                for nro_ite in range(CORRIDAS):
                    # Establece resultado esperado según la clase. 1=clase esperada, 0=otra clase
                
                    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, T, test_size=TEST_SIZE)#, random_state=42)
                    # Entrena perceptrón
                    modelo = NeuronaGradiente(alpha=ALFA, n_iter=MAX_ITE, FUN=FUN, COSTO=COSTO, cotaE=ERROR)
                    modelo.fit(X_train, Y_train)
                
                    (W, b, iteraciones) = (modelo.w_, modelo.b_, len(modelo.errors_))
                
                    prom_ite = prom_ite + iteraciones
                
                    # efectividad entrenamiento
                    Y_pred = modelo.predict(X_train)
                    efectividad = 100.0*np.sum(Y_train==Y_pred)/len(Y_pred)
                    prom_acc_train = prom_acc_train + efectividad
                    if (100-efectividad>0.0000001):
                        veces_train = veces_train + 1
                
                    # efectividad entrenamiento
                    Y_pred = modelo.predict(X_test)
                    efectividad = 100.0*np.sum(Y_test==Y_pred)/len(Y_pred)
                    prom_acc_test = prom_acc_test + efectividad
                    if (100-efectividad>0.0000001):
                      veces_test = veces_test + 1
                
                    # print("Clase %s con %6.2f%% de efectividad con %d iteraciones" % (cls_name, efectividad, iteraciones ))
                TABLA.append([FUN, ALFA, COSTO, TEST_SIZE, (prom_ite/CORRIDAS), (prom_acc_train/CORRIDAS), veces_train, (prom_acc_test/CORRIDAS), veces_test])
                # print(f'30 ejec con {FUN} y con alfa = {ALFA} ejecutadas correctamente')


print('Codigo ejecutado correctamente')

/home/juan/Documents/Facultad/Deep Learning/Practica 3B - Neurona no Lineal/ClassNeuronaGral.py:105: RuntimeWarning: invalid value encountered in log
  return(-y*np.log(y_hat+EPS)-(1-y)*np.log(1-y_hat+EPS))


Codigo ejecutado correctamente


In [11]:
from tabulate import tabulate
table = tabulate(TABLA, headers="firstrow", tablefmt="grid")
print(table)

+-----------+--------+------------+-------------+---------------+------------------+-----------------------+-----------------+----------------------+
| Funcion   |   Alfa | Costo      |   TEST_SIZE |   Iteraciones |   Accuracy train |   Veces (datos Train) |   Accuracy test |   Veces (Datos Test) |
+===========+========+============+=============+===============+==================+=======================+=================+======================+
| sigmoid   |    0.1 | ECM        |         0.5 |      126      |         100      |                     0 |         98.9888 |                   12 |
+-----------+--------+------------+-------------+---------------+------------------+-----------------------+-----------------+----------------------+
| sigmoid   |    0.1 | ECM        |         0.6 |      113.4    |         100      |                     0 |         98.1931 |                   25 |
+-----------+--------+------------+-------------+---------------+------------------+----------------